# CCDC Analysis - Usage Examples

This notebook demonstrates how to use the `ccdc_analysis` module for easy CCDC analysis.

In [1]:
# Import required libraries
import ee
import pandas as pd
import matplotlib.pyplot as plt

# Import the CCDC analysis module
from ccdc_analysis import run_ccdc_analysis, export_ccdc_results

# Initialize Earth Engine
ee.Initialize()


ModuleNotFoundError: No module named 'ccdc_analysis'

## Example 1: Basic Usage - Single Point Analysis

Run CCDC analysis for a single point with minimal parameters.

In [ ]:
# Define parameters
lat = -10.5
lon = -50.2
start_year = 2000
end_year = 2022
target_year = 2010

# Run analysis
result = run_ccdc_analysis(
    lat=lat,
    lon=lon,
    start_year=start_year,
    end_year=end_year,
    target_year=target_year,
    point_id=1,
    use_false_color=False  # True color RGB
)

# Display the plot
plt.show()


## Example 2: Using Data from CSV File

Load point data from CSV and run analysis for a specific point.

In [ ]:
# Load validation points
csv_file = '/projectnb/modislc/users/chishan/data/MapBiomas/MAPBIOMAS/mapbiomas_85k_col2_1_points_english.csv'
df_points = pd.read_csv(csv_file)

# Select a specific point
point_id = 37464
point_data = df_points[df_points['TARGETID'] == point_id].iloc[0]

# Extract parameters
lat = point_data['LAT']
lon = point_data['LON']
biome = point_data.get('BIOMA_250K', 'Unknown')

# Run analysis with all metadata
result = run_ccdc_analysis(
    lat=lat,
    lon=lon,
    start_year=2000,
    end_year=2022,
    target_year=2006,
    point_id=point_id,
    biome=biome,
    use_false_color=True  # False color (NIR-R-G)
)

plt.show()


## Example 3: Batch Processing Multiple Points

Process multiple points and save results to files.

In [ ]:
# Load validation points
csv_file = '/projectnb/modislc/users/chishan/data/MapBiomas/MAPBIOMAS/mapbiomas_85k_col2_1_points_english.csv'
df_points = pd.read_csv(csv_file)

# Select points to process (e.g., first 5 points)
points_to_process = df_points.head(5)

# Output directory
output_dir = '/usr2/postdoc/chishan/analysis_results/ccdc_batch'

# Process each point
for idx, row in points_to_process.iterrows():
    point_id = row['TARGETID']
    lat = row['LAT']
    lon = row['LON']
    biome = row.get('BIOMA_250K', 'Unknown')
    
    print(f"\nProcessing Point {point_id}...")
    
    try:
        result = run_ccdc_analysis(
            lat=lat,
            lon=lon,
            start_year=2000,
            end_year=2022,
            target_year=2010,
            point_id=point_id,
            biome=biome,
            use_false_color=False,
            verbose=True
        )
        
        # Export results
        export_ccdc_results(result, output_dir, point_id)
        
        # Close the figure to free memory
        plt.close(result['fig'])
        
    except Exception as e:
        print(f"Error processing point {point_id}: {e}")

print(f"\nBatch processing complete! Results saved to: {output_dir}")


## Example 4: Access and Analyze Results

Work with the returned data for custom analysis.

In [ ]:
# Run analysis
result = run_ccdc_analysis(
    lat=-10.5,
    lon=-50.2,
    start_year=2000,
    end_year=2022,
    target_year=2010,
    point_id=999
)

# Access the time series DataFrame
df = result['df_ccdc']
print("\nTime Series Data:")
print(df.head())

# Access segment information
segments = result['segments']
print(f"\nNumber of segments: {len(segments)}")
for idx, start_dt, end_dt in segments:
    print(f"Segment {idx + 1}: {start_dt.date()} to {end_dt.date()}")

# Calculate custom statistics
print("\nOverall Statistics:")
print(f"Mean NDVI: {df['ndvi'].mean():.3f}")
print(f"Std NDVI: {df['ndvi'].std():.3f}")
print(f"RMSE: {((df['ndvi'] - df['fit']) ** 2).mean() ** 0.5:.3f}")

# Save to custom location
df.to_csv('/usr2/postdoc/chishan/my_ccdc_results.csv', index=False)
result['fig'].savefig('/usr2/postdoc/chishan/my_ccdc_plot.png', dpi=300, bbox_inches='tight')

plt.show()


## Example 5: Customize CCDC Parameters

Fine-tune CCDC algorithm parameters for specific use cases.

In [ ]:
# Run with custom CCDC parameters
result = run_ccdc_analysis(
    lat=-10.5,
    lon=-50.2,
    start_year=2000,
    end_year=2022,
    target_year=2010,
    point_id=1234,
    # Imagery options
    use_false_color=True,
    buffer_distance=2000,  # 2km buffer for imagery
    cloud_threshold=20,    # Maximum 20% cloud cover
    # CCDC algorithm parameters
    ccdc_lambda=15/10000.0,  # More sensitive to changes
    ccdc_max_iter=15000,     # More iterations
    min_observations=8,      # Require more observations per segment
    chi_square_prob=0.95,    # Different confidence level
    verbose=True
)

plt.show()
